<a href="https://colab.research.google.com/github/futugyou/pyproject/blob/master/google_colab/generation_embedding_model_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required dependencies
%pip install datasets
%pip install sentence_transformers
%pip install transformers
%pip install torch
%pip install tqdm
%pip install scikit-learn
%pip install mteb

In [ ]:
from datasets import load_dataset, Dataset
import random
from tqdm import tqdm

mnli = load_dataset("glue", "mnli", split="train").select(range(50_000))
mnli = mnli.remove_columns("idx")
mnli = mnli.filter(lambda x: True if x["label"] == 0 else False)

train_dataset = {"anchor": [], "positive": [], "negative": []}
soft_negatives = mnli["hypothesis"]
random.shuffle(soft_negatives)
for row, soft_negative in tqdm(zip(mnli, soft_negatives), total=len(mnli)):
    train_dataset["anchor"].append(row["premise"])
    train_dataset["positive"].append(row["hypothesis"])
    train_dataset["negative"].append(soft_negative)
train_dataset = Dataset.from_dict(train_dataset)
train_dataset[2]

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

val_sts = load_dataset("glue", "stsb", split="validation")
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score / 5 for score in val_sts["label"]],
    main_similarity="cosine",
)

In [ ]:
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# model
mebedding_model = SentenceTransformer("bert-base-uncased")

# loss function
train_loss = losses.MultipleNegativesRankingLoss(model=mebedding_model)

# training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="mrloss_mebedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to="none",
)

In [ ]:
# train model
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator,
)
trainer.train()

In [ ]:
# evaluator model
evaluator(embedding_model)

In [ ]:
# MTEB evaluator, it costs a lot of time to run

from mteb import MTEB

evaluator = MTEB(tasks=["Banking77Classification"])
resuls = evaluator.run(embedding_model)